In [58]:
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Knupf\PycharmProjects\chatbot_project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [59]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Knupf\PycharmProjects\chatbot_project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [60]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Knupf\PycharmProjects\chatbot_project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [61]:
from elastic_search.utilities.login import login
from utilities.es_search import search_es
from transformers import pipeline
import json

# Variables

In [62]:
# Get index name
with open('..\\elastic_search\\utilities\\index_config.json') as file:
    index_name = json.load(file)[0]['index_name']

# Connect to elastic search

In [63]:
es = login()

Connecting to elastic search...
Connection to elastic search successfully established


C:\Users\Knupf\PycharmProjects\chatbot_project\venv\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


# Search in index

In [64]:

question_text = 'Was steht im Fokus des Studiums?'

es_answers = search_es(es_instance=es, index_name=index_name, question_text=question_text, number_of_results=10)

print(f'Question: {question_text}')
print(f'Query Duration: {es_answers["took"]} milliseconds')
print('Title, Relevance Score:')
[(hit['_source']['document_title'], hit['_score']) for hit in es_answers['hits']['hits']]


Question: Was steht im Fokus des Studiums?
Query Duration: 2 milliseconds
Title, Relevance Score:


C:\Users\Knupf\PycharmProjects\chatbot_project\venv\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[('Wirtschaftsinformatik-Application Management', 5.001475),
 ('Studienziele', 3.2230263),
 ('Studienportal', 2.1792238),
 ('Studienvorbereitung Mathematik', 2.1108859),
 ('Studieninhalte', 1.4140074),
 ('Studienbeginn & Regelstudienzeit', 1.1231436),
 ('Eckdaten Wirtschaftsinformatik-Application Management', 0.7666509),
 ('Job & Karriere', 0.65873516)]

In [65]:
qas = pipeline(
    model='deutsche-telekom/bert-multi-english-german-squad2',
    task='question-answering'
    )

In [66]:
answer = qas(
    question=question_text,
    context=es_answers['hits']['hits'][0]['_source']['document_text']
    )
print(answer)

{'score': 0.07189861685037613, 'start': 262, 'end': 286, 'answer': 'aktuelle IT-Technologien'}


In [67]:
answer_dict = dict([
    ("question", question_text),
    ("answer", answer["answer"]),
    ("score", answer['score']),
    ('link', es_answers['hits']['hits'][0]['_source']['link'])
    ])

with open('answer.json', 'w') as file:
    json.dump(answer_dict, file, ensure_ascii=False)
